In [1]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
from torch.optim import SGD
from experiment_setup import MNIST_Experiment

In [2]:
ell = nn.CrossEntropyLoss()
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
epochs  = 40
ValAccs = []
reps    = 10

for rep in range(reps):
    conf = MNIST_Experiment('params/mirror.yaml',data_path = "../../../../datasets/")
    model = conf.model.to(device)
    opt = SGD(model.parameters())
    for e in range(epochs):
        acc = 0
        n = 0
        for x,y in iter(conf.train_loader):
            x,y = x.to(device), y.to(device)
            opt.zero_grad()
            logit = model(x)
            loss = ell(logit, y)
            loss.backward()
            opt.step()
    
            acc += (logit.topk(1)[1].squeeze() == y).sum()
            n   += y.shape[0]
        print('Train Acc: ' +str(acc.item()/n))
    
    print(30*'<>')
    print('Finished run ' + str(rep))
    with torch.no_grad():
        acc = 0
        n = 0
        model.eval()
        for x,y in iter(conf.test_loader):
            x,y = x.to(device), y.to(device)
            logit = model(x)
            acc += (logit.topk(1)[1].squeeze() == y).sum()
            n   += y.shape[0]
        print('Val Acc: ' +str(acc.item()/n))
        ValAccs.append(acc.item()/n)

In [ ]:
print('Mean Acc: ' + str(100 * torch.tensor(ValAccs).mean().item()))